In [1]:
import gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [2]:

from gym.wrappers import Monitor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import pow, add, mul, div, sqrt
from torch.utils.tensorboard import SummaryWriter
from tensorboard import program

In [4]:
import os
import math
import copy
import numpy as np
import datareader
from time_recorder import time_keeper
from FastDataLoader import FastTensorDataLoader
from sklearn.model_selection import train_test_split
from plotting_functions import compare_spectra

In [5]:
def read_data( x_range, y_range, geoboundary,  batch_size=128,
                 data_dir=os.path.abspath(''), rand_seed=1234, normalize_input = True, test_ratio = 0.999):
    """
      :param input_size: input size of the arrays
      :param output_size: output size of the arrays
      :param x_range: columns of input data in the txt file
      :param y_range: columns of output data in the txt file
      :param cross_val: number of cross validation folds
      :param val_fold: which fold to be used for validation
      :param batch_size: size of the batch read every time
      :param shuffle_size: size of the batch when shuffle the dataset
      :param data_dir: parent directory of where the data is stored, by default it's the current directory
      :param rand_seed: random seed
      :param test_ratio: if this is not 0, then split test data from training data at this ratio
                         if this is 0, use the dataIn/eval files to make the test set
      """

    # Import data files
    print('Importing data files...')

    ftrTrain, lblTrain = datareader.importData(os.path.join(data_dir, 'dataIn'), x_range, y_range)
    if (test_ratio > 0):
        print("Splitting data into training and test sets with a ratio of:", str(test_ratio))
        ftrTrain, ftrTest, lblTrain, lblTest = train_test_split(ftrTrain, lblTrain,
                                                                test_size=test_ratio, random_state=rand_seed)
        print('Total number of training samples is {}'.format(len(ftrTrain)))
        print('Total number of test samples is {}'.format(len(ftrTest)))
        print('Length of an output spectrum is {}'.format(len(lblTest[0])))
    else:
        print("Using separate file from dataIn/Eval as test set")
        ftrTest, lblTest = datareader.importData(os.path.join(data_dir, 'dataIn', 'eval'), x_range, y_range)

    # print('Total number of training samples is {}'.format(len(ftrTrain)))
    # print('Total number of test samples is {}'.format(len(ftrTest)))
    # print('Length of an output spectrum is {}'.format(len(lblTest[0])))
    # print('downsampling output curves')
    # resample the output curves so that there are not so many output points
    if len(lblTrain[0]) > 2000:                                 # For Omar data set
        lblTrain = lblTrain[::, len(lblTest[0])-1800::6]
        lblTest = lblTest[::, len(lblTest[0])-1800::6]

    # print('length of downsampled train spectra is {} for first, {} for final, '.format(len(lblTrain[0]),
    #                                                                                    len(lblTrain[-1])),
    #       'set final layer size to be compatible with this number')

    print('Generating torch datasets')
    assert np.shape(ftrTrain)[0] == np.shape(lblTrain)[0]
    assert np.shape(ftrTest)[0] == np.shape(lblTest)[0]

    # Normalize the data if instructed using boundary
    if normalize_input:
        ftrTrain[:,0:4] = (ftrTrain[:,0:4] - (geoboundary[0] + geoboundary[1]) / 2)/(geoboundary[1] - geoboundary[0]) * 2
        ftrTest[:,0:4] = (ftrTest[:,0:4] - (geoboundary[0] + geoboundary[1]) / 2)/(geoboundary[1] - geoboundary[0]) * 2
        ftrTrain[:,4:] = (ftrTrain[:,4:] - (geoboundary[2] + geoboundary[3]) / 2)/(geoboundary[3] - geoboundary[2]) * 2
        ftrTest[:,4:] = (ftrTest[:,4:] - (geoboundary[2] + geoboundary[3]) / 2)/(geoboundary[3] - geoboundary[2]) * 2

    # train_data = datareader.MetaMaterialDataSet(ftrTrain, lblTrain, bool_train= True)
    # test_data = datareader.MetaMaterialDataSet(ftrTest, lblTest, bool_train= False)
    # train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    # test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

    return ftrTest, lblTest


In [6]:
import flagreader
flags = flagreader.read_flag()
batch_size = 4096
ftrTest, lblTest = read_data(x_range=flags.x_range,
                                                     y_range=[i for i in range(20 , 320 )],
                                                     geoboundary=flags.geoboundary,
                                                     batch_size=batch_size,
                                                     normalize_input=flags.normalize_input,
                                                     data_dir=flags.data_dir,
                                                     test_ratio=0.999)

Importing data files...
['ToyModelSim_e2_with_params_16.csv', 'ToyModelSim_e2_with_params_3.csv', 'ToyModelSim_e2_with_params_10.csv', 'ToyModelSim_e2_with_params_12.csv', 'ToyModelSim_e2_with_params_15.csv', 'ToyModelSim_e2_with_params_11.csv', 'ToyModelSim_e2_with_params_4.csv', 'ToyModelSim_e2_with_params_23.csv', 'ToyModelSim_e2_with_params_7.csv', 'ToyModelSim_e2_with_params_25.csv', 'ToyModelSim_e2_with_params_9.csv', 'ToyModelSim_e2_with_params_8.csv', 'ToyModelSim_e2_with_params_6.csv', 'ToyModelSim_e2_with_params_13.csv', 'ToyModelSim_e2_with_params_2.csv', 'ToyModelSim_e2_with_params_24.csv', 'ToyModelSim_e2_with_params_22.csv', 'ToyModelSim_e2_with_params_5.csv', 'ToyModelSim_e2_with_params_1.csv', 'ToyModelSim_e2_with_params_18.csv', 'ToyModelSim_e2_with_params_17.csv', 'ToyModelSim_e2_with_params_19.csv', 'ToyModelSim_e2_with_params_21.csv', 'ToyModelSim_e2_with_params_20.csv', 'ToyModelSim_e2_with_params_14.csv']
Splitting data into training and test sets with a ratio of:

In [7]:
def reload_data(ftrTest,lblTest):
    # train_loader = FastTensorDataLoader(torch.from_numpy(ftrTrain),
    #                                     torch.from_numpy(lblTrain), batch_size=batch_size, shuffle=True)
    test_loader = FastTensorDataLoader(torch.from_numpy(ftrTest),
                                       torch.from_numpy(lblTest), batch_size=batch_size, shuffle=True)

    return test_loader

In [8]:
class LorentzNN(nn.Module):
    def __init__(self, flags):
        super(LorentzNN, self).__init__()
        self.flags = flags

        w_numpy = np.arange(flags.freq_low, flags.freq_high,
                            (flags.freq_high - flags.freq_low) / self.flags.num_spec_points)

        cuda = True if torch.cuda.is_available() else False
        if cuda:
            self.w = torch.tensor(w_numpy).cuda()
        else:
            self.w = torch.tensor(w_numpy)

        self.linears = nn.ModuleList([])
        for ind, fc_num in enumerate(flags.linear[0:-1]):
            self.linears.append(nn.Linear(fc_num, flags.linear[ind + 1], bias=True))

        layer_size = flags.linear[-1]
        self.lin_w0 = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)
        self.lin_wp = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)
        self.lin_g = nn.Linear(layer_size, self.flags.num_lorentz_osc, bias=False)

    def forward(self, G):
        out = G
        for ind, fc in enumerate(self.linears):
            if ind <= len(self.linears):
                out = F.relu(fc(out))

        w0 = F.relu(self.lin_w0(out))
        wp = F.relu(self.lin_wp(out))
        g = F.relu(self.lin_g(out))

        w0 = w0.unsqueeze(2) * 1
        wp = wp.unsqueeze(2) * 1
        g = g.unsqueeze(2) * 0.1

        w0 = w0.expand(out.size(0), self.flags.num_lorentz_osc, self.flags.num_spec_points)
        wp = wp.expand_as(w0)
        g = g.expand_as(w0)
        w_expand = self.w.expand_as(g)

        e2 = div(mul(pow(wp, 2), mul(w_expand, g)),
                 add(pow(add(pow(w0, 2), -pow(w_expand, 2)), 2), mul(pow(w_expand, 2), pow(g, 2))))

        e2 = torch.sum(e2, 1)
        return e2.float()

In [9]:
dir = '/home/omar/PycharmProjects/mlmOK_Pytorch/'
tk = time_keeper(time_keeping_file=os.path.join(dir, 'training_time.txt'))

In [10]:
def init_weights(m):

        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters

        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features

        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            if m.bias:
                m.bias.data.fill_(0.00)

In [11]:
def return_random_agents(num_agents):

    agents = []
    for _ in range(num_agents):

        agent = LorentzNN(flags)

        for param in agent.parameters():
            param.requires_grad = False

        init_weights(agent)
        agents.append(agent)


    return agents

In [12]:
def run_agents(agents):

    reward_agents = []

    for agent_num, agent in enumerate(agents):
        cuda = True if torch.cuda.is_available() else False
        if cuda:
            agent.cuda()
        agent.eval()

        eval_loss = []
        with torch.no_grad():
            for ind, (geometry, spectra) in enumerate(test_loader):
                if cuda:
                    geometry = geometry.cuda()
                    spectra = spectra.cuda()
                if (batch_size*(ind+1) < 20000):
                    logit = agent(geometry)
                    loss = nn.functional.mse_loss(logit, spectra)
                    eval_loss.append(np.copy(loss.cpu().data.numpy()))
                else:
                    break
        eval_avg_loss = np.mean(eval_loss)
        # reward = 1/eval_avg_loss
        # reward = math.exp(reward)-1
        reward = -eval_avg_loss
        if math.isnan(reward):
            reward = 0
        reward_agents.append(reward)
        # print(agent_num)

    return reward_agents

In [13]:
def return_average_score(agent, runs):
    score = 0.
    for i in range(runs):
        score += run_agents([agent])[0]
    return score/runs

In [14]:
def run_agents_n_times(agents, runs):
    avg_score = []
    for agent in agents:
        avg_score.append(return_average_score(agent,runs))
    return avg_score

In [15]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)

    mutation_power = 0.05 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf
    sparsity_index = 0.2
    # for param in child_agent.parameters():
    for layer_name, child in child_agent.named_children():
        for param in child.parameters():
            if(layer_name == 'lin_w0' or layer_name == 'lin_wp'
                    or layer_name == 'lin_g'):
                mutation_tensor = torch.randn_like(param)
                mut_sorted,ind = torch.sort(mutation_tensor.view(-1,1))
                limit = int((len(mut_sorted)-1)*(1-sparsity_index))
                mutation_tensor[torch.abs(mutation_tensor) < mut_sorted[limit]] = 0
                param += torch.randn_like(param) * mutation_power
            else:
                param += torch.randn_like(param) * mutation_power

                # print(param)

        # if(len(param.shape)==4): #weights of Conv2D
        #     # a=1
        #     for i0 in range(param.shape[0]):
        #         for i1 in range(param.shape[1]):
        #             for i2 in range(param.shape[2]):
        #                 for i3 in range(param.shape[3]):
        #
        #                     param[i0][i1][i2][i3]+= mutation_power * np.random.randn()
        #
        #
        #
        # elif(len(param.shape)==2): #weights of linear layer
        #     # param += np.random.rand(param.shape[0],param.shape[1]) * mutation_power
        #
        #     for i0 in range(param.shape[0]):
        #         for i1 in range(param.shape[1]):
        #
        #             param[i0][i1]+= mutation_power * np.random.randn()
        #
        #
        # elif(len(param.shape)==1): #biases of linear layer or conv layer
        #     for i0 in range(param.shape[0]):
        #
        #         param[i0]+=mutation_power * np.random.randn()

    return child_agent

In [16]:
def return_children(agents, sorted_parent_indexes, elite_index):

    children_agents = []

    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):

        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one

    return children_agents, elite_index


In [17]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):

    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]

    if(elite_index is not None):
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])

    top_score = None
    top_elite_index = None

    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=1)
        print("Score for elite i ", i, " is ", score)

        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i

    print("Elite selected with index ",top_elite_index, " and score", top_score)

    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent


In [18]:
def start_tensorboard(model_path):

    tb = program.TensorBoard()
    log = SummaryWriter(model_path)
    tb.configure(argv=[None, '--logdir', model_path])

    return tb

In [23]:
def plot_elite(agent, log, gen):
    cuda = True if torch.cuda.is_available() else False
    if cuda:
        agent.cuda()
    agent.eval()

    eval_loss = []
    with torch.no_grad():
        for ind, (geometry, spectra) in enumerate(test_loader):
            if cuda:
                geometry = geometry.cuda()
                spectra = spectra.cuda()
            if (batch_size*(ind+1) < 20000):
                logit = agent(geometry)
                loss = nn.functional.mse_loss(logit, spectra)
                eval_loss.append(np.copy(loss.cpu().data.numpy()))
            else:
                break
    eval_avg_loss = -np.mean(eval_loss)
    log.add_scalar('Loss', eval_avg_loss, gen)
    for k in range(10):
        f = compare_spectra(Ypred=logit[k, :].cpu().data.numpy(), Ytruth=spectra[k, :].cpu().data.numpy(),
                            xmin=flags.freq_low, xmax=flags.freq_high, num_points=flags.num_spec_points)
        log.add_figure(tag='Test ' + str(k) +') Sample e2 Spectrum'.format(1),
                                                    figure=f, global_step=gen)

In [24]:
# game_actions = 2 #2 actions possible: left or right

#disable gradients as we will not use them
torch.set_grad_enabled(False)

# initialize N number of agents
num_agents = 500
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

# run evolution until X generations
generations = 5

elite_index = None

model_path = '/home/omar/PycharmProjects/mlmOK_Pytorch/models/GA'
tb = program.TensorBoard()
log = SummaryWriter(model_path)
tb.configure(argv=[None, '--logdir', model_path])
url = tb.launch()
print("TensorBoard started at %s" % url)

for generation in range(generations):

    test_loader = reload_data(ftrTest,lblTest)
    # return rewards of agents
    # rewards = run_agents_n_times(agents, 1) #return average of 3 runs

    rewards = run_agents(agents)

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order

    print("")
    print("")

    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])

    print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
    #print(rewards)
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",top_rewards)

    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    plot_elite(children_agents[elite_index],log, generation)

    # kill all agents, and replace them with their children
    agents = children_agents


TensorBoard started at http://localhost:6008/


Generation  0  | Mean rewards:  -56.902763  | Mean of top 5:  -56.041283
Top  20  scores [153 433 120 309  67  12  87 168 265 210 489 371 442 152 419 227 480  53
 115 357]
Rewards for top:  [-55.927677, -56.002205, -56.03347, -56.099392, -56.143684, -56.23887, -56.259586, -56.261337, -56.26413, -56.266647, -56.296715, -56.298393, -56.304348, -56.318085, -56.327118, -56.32731, -56.34575, -56.348694, -56.348904, -56.35109]
Score for elite i  153  is  -57.0632438659668
Score for elite i  433  is  -57.08674621582031
Score for elite i  120  is  -57.16672134399414
Score for elite i  309  is  -56.91170883178711
Score for elite i  67  is  -56.839237213134766
Score for elite i  12  is  -57.308815002441406
Score for elite i  87  is  -56.61870193481445
Score for elite i  168  is  -56.44713592529297
Score for elite i  265  is  -57.109867095947266
Score for elite i  210  is  -56.66191864013672
Elite selected with index  168  and score -56.447135925292

KeyboardInterrupt: 